In [19]:
from Scripts.import_data import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
import numpy as np

Zaimportowanie wartości rynkowej

In [20]:
#market_values to procentowe roczne zmiany wartości spółek, obliczone w notebooku "Market_value_importer"
market_values = pd.read_csv('./Data/market_values.csv')
market_values.set_index('Unnamed: 0', inplace=True)
print(market_values.index)
print(market_values)

Index([    'PKOBP',  'PKNORLEN',       'PZU',     'SANPL', 'CDPROJEKT',
           'PEKAO',    'INGBSK',     'PGNIG',      'KGHM', 'CYFRPLSAT',
       ...
         'COMPLEX', 'MOSTALEXP',    'MISPOL',   'IDEATFI', 'BUDVARCEN',
       'GLOBALNRG',  'GREENECO', 'ENERGOPLD',    'CCIINT',   'NOVAKBM'],
      dtype='object', name='Unnamed: 0', length=633)
                 2022       2021       2020          2019       2018  \
Unnamed: 0                                                             
PKOBP       56.441504 -16.656994 -12.693185    -10.923042  57.462687   
PKNORLEN    28.794157 -32.742939 -20.647244      2.028292  24.267301   
PZU          9.239826 -19.160652  -8.815479      4.127135  26.949710   
SANPL       87.668260 -39.530316 -14.182033  36568.030000        NaN   
CDPROJEKT  -29.719242   2.919316  91.964313     50.103071  85.823903   
...               ...        ...        ...           ...        ...   
GLOBALNRG         NaN        NaN        NaN           NaN        NaN   

Zaimportowanie danych dotyczących alokacji środków

In [21]:
balance_data = import_csv_dict_of_dataframes('./Data/companies_balance_sheet.csv')

for company, years in balance_data.items():
    for year, data in years.items():
        if company in market_values.index and year in market_values.columns:
            # Take only the second level of the MultiIndex
            data.index = data.index.droplevel(0)
balance_data

{'11BIT':                                                                 2017  \
 11BIT End of period                                       2017-12-31   
       ASSETS                                               48649.817   
       Non-current assets                                   17173.693   
       Property, plant and equipment                          814.332   
       Exploration for and evaluation of mineral resou...         NaN   
 ...                                                              ...   
       Provisions                                                 NaN   
       Other liabilities                                          NaN   
       Accruals (liability)                                   466.396   
       Liabilities related to assets held for sale and...         NaN   
       Date of publication                                 2019-03-27   
 
                                                                         2018  \
 11BIT End of period            

Stworzenie jednego zbioru z wszystkimi danymi

In [22]:
columns = ['ASSETS','Non-current assets','Property, plant and equipment','Exploration for and evaluation of mineral resources','Intangible assets','Goodwill','Investment property',
'Right-of-use assets','Investment in affiliates','Non-current financial assets','Non-current loans and receivables','Deferred income tax','Non-current deferred charges and accruals',
'Non-current derivative instruments','Other non-current assets','Current assets','Inventories','Current intangible assets',
'Biological assets','Trade receivables','Loans and other receivables','Financial assets','Cash and cash equivalents','Accruals','Assets from current tax','Derivative instruments',
'Other assets','Assets held for sale and discontinuing operations','Called up capital','Own shares','EQUITY & LIABILITIES','Equity shareholders of the parent','Share capital',
'Called up share capital','Treasury shares','Supplementary capital','Valuation and exchange differences','Other capitals','Retained earnings / accumulated losses','Non-controlling interests',
'Non-current liabilities','Non-current liabilities from derivatives','Non-current loans and borrowings','Non-current liabilities from bonds','Non-current liabilities from finance leases',
'Non-current trade payables','Long-term provision for employee benefits','Deferred tax liabilities','Non-current provision','Other non-current liabilities','Non-current accruals (liability)',
'Current liabilities','Liabilities from derivatives','Financial liabilities (loans and borrowings)','Bond liabilities','Liabilities from finance leases','Trade payables','Employee benefits',
'Current tax liabilities','Provisions','Other liabilities','Accruals (liability)','Liabilities related to assets held for sale and discontinued operations', 'market_value']

rows = []
for company, years in balance_data.items():
    for year, market_data in years.items():
        new_row = {}
        for indicator, value in market_data.items():
            if indicator in columns:
                new_row[indicator] = value
        if company in market_values.index and year in market_values.columns:
            new_row['market_value'] = market_values[year][company]
            rows.append(new_row)

data = pd.DataFrame(rows, columns=columns)
data

,ASSETS,Non-current assets,"Property, plant and equipment",Exploration for and evaluation of mineral resources,Intangible assets,Goodwill,Investment property,Right-of-use assets,Investment in affiliates,Non-current financial assets,...,Bond liabilities,Liabilities from finance leases,Trade payables,Employee benefits,Current tax liabilities,Provisions,Other liabilities,Accruals (liability),Liabilities related to assets held for sale and discontinued operations,market_value
0,48649.817,17173.693,814.332,NaN,13067.498,NaN,NaN,NaN,NaN,3053.534,...,NaN,NaN,1197.853,NaN,NaN,NaN,NaN,466.396,NaN,107.934581
1,109262.821,41316.893000000004,18734.064000000002,NaN,17138.011,NaN,NaN,NaN,NaN,5094.282,...,NaN,NaN,5122.519,NaN,NaN,NaN,NaN,758.971,NaN,39.139118
2,155668.388,49815.126000000004,20380.205,NaN,24870.505,NaN,NaN,4007.699,NaN,NaN,...,NaN,12.653,5494.21,NaN,556.548,NaN,41.795,17256.979,NaN,23.231833
3,186339.222,70397.597,26889.503,NaN,37859.517,NaN,NaN,4003.398,NaN,NaN,...,NaN,12.651,7444.283,NaN,1184.903,860.272,41.796,320.698,NaN,63.933486
4,219512.964,90679.674,25946.479,NaN,59137.637,NaN,NaN,4281.361,NaN,NaN,...,NaN,14.734,6332.759,NaN,744.927,661.422,41.796,4169.079,NaN,22.300192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,1778284,994505,809413,NaN,36338,NaN,NaN,NaN,58102,NaN,...,NaN,NaN,913510,4258,39660,10012,NaN,NaN,NaN,14.618363
1241,1790063,1014534,818103,NaN,29100,NaN,NaN,NaN,58102,NaN,...,NaN,NaN,1011972,2327,2528,11769,NaN,NaN,NaN,6.546306
1242,2479535,1595330,905788,NaN,26674,NaN,NaN,35897,532325,NaN,...,NaN,6488,704176,628,76761,8900,NaN,321194,NaN,-2.118575
1243,2513606,1729256,1053811,NaN,360038,NaN,NaN,83331,58102,NaN,...,NaN,7175,586113,204,103458,11302,NaN,350508,NaN,6.060657


Czyszczenie danych

In [23]:
data = data.fillna(0)
data = data.apply(pd.to_numeric, errors='coerce')
data = data[data['ASSETS'] > 0]
y = data['market_value']
X = data.drop('market_value', axis=1)
X

,ASSETS,Non-current assets,"Property, plant and equipment",Exploration for and evaluation of mineral resources,Intangible assets,Goodwill,Investment property,Right-of-use assets,Investment in affiliates,Non-current financial assets,...,Financial liabilities (loans and borrowings),Bond liabilities,Liabilities from finance leases,Trade payables,Employee benefits,Current tax liabilities,Provisions,Other liabilities,Accruals (liability),Liabilities related to assets held for sale and discontinued operations
0,48649.817,17173.693,814.332,0,13067.498,0.0,0.0,0.000,0.0,3053.534,...,0.000,0.0,0.000,1197.853,0.0,0.000,0.000,0.000,466.396,0.0
1,109262.821,41316.893,18734.064,0,17138.011,0.0,0.0,0.000,0.0,5094.282,...,1452.203,0.0,0.000,5122.519,0.0,0.000,0.000,0.000,758.971,0.0
2,155668.388,49815.126,20380.205,0,24870.505,0.0,0.0,4007.699,0.0,0.000,...,1303.717,0.0,12.653,5494.210,0.0,556.548,0.000,41.795,17256.979,0.0
3,186339.222,70397.597,26889.503,0,37859.517,0.0,0.0,4003.398,0.0,0.000,...,1260.000,0.0,12.651,7444.283,0.0,1184.903,860.272,41.796,320.698,0.0
4,219512.964,90679.674,25946.479,0,59137.637,0.0,0.0,4281.361,0.0,0.000,...,1260.000,0.0,14.734,6332.759,0.0,744.927,661.422,41.796,4169.079,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,1778284.000,994505.000,809413.000,0,36338.000,0.0,0.0,0.000,58102.0,0.000,...,97060.000,0.0,0.000,913510.000,4258.0,39660.000,10012.000,0.000,0.000,0.0
1241,1790063.000,1014534.000,818103.000,0,29100.000,0.0,0.0,0.000,58102.0,0.000,...,418770.000,0.0,0.000,1011972.000,2327.0,2528.000,11769.000,0.000,0.000,0.0
1242,2479535.000,1595330.000,905788.000,0,26674.000,0.0,0.0,35897.000,532325.0,0.000,...,71704.000,0.0,6488.000,704176.000,628.0,76761.000,8900.000,0.000,321194.000,0.0
1243,2513606.000,1729256.000,1053811.000,0,360038.000,0.0,0.0,83331.000,58102.0,0.000,...,64084.000,0.0,7175.000,586113.000,204.0,103458.000,11302.000,0.000,350508.000,0.0


In [24]:
# zamiana na procenty
X_normalized = X.copy()

# ASSETS
columns_to_divide = X.columns[1:30]
X_normalized.iloc[:, 1:30] = X_normalized.iloc[:, 1:30].div(X['ASSETS'], axis=0)

In [25]:
# Detekcja i usuwanie anomalii

def delete_anomaly(X: pd.DataFrame, y: pd.DataFrame, threshold: float) -> pd.DataFrame:
    # Calculate the mean for each column except the first one
    mean_val = X[X.columns[1:30]].mean()
    
    # Calculate the absolute deviation from the mean for each column except the first one
    deviation = X[X.columns[1:30]].sub(mean_val).abs()

    # Create a boolean DataFrame where each element is True if it meets the condition
    condition = (deviation <= threshold)
    # Create a mask for rows where all columns meet the condition
    row_mask = condition.all(axis=1)
    # Filter the DataFrame using the row mask
    X_filtered = X[row_mask]
    y_filtered = y[row_mask]
    
    return X_filtered, y_filtered

In [26]:
X_norm, y_norm = delete_anomaly(X_normalized, y, 0.2)

Uczenie modelu

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Support Vector Machine': SVR(kernel='linear'),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} MSE: {mse}')

Linear Regression MSE: 34408451583739.58
Random Forest MSE: 5834.022964641257
Gradient Boosting MSE: 10938.129067979507
